# Word Counts (seperated by spaces)

In [ ]:
from dataset import *
data = Dataset()

In [7]:
tamil_train = pd.read_csv(data.fire_2022_tam_train, '\t')
tamil_val = pd.read_csv(data.fire_2022_tam_val, '\t')

kan_train = pd.read_csv(data.fire_2022_kan_train, '\t')
kan_val = pd.read_csv(data.fire_2022_kan_val, '\t')

mal_train = pd.read_csv(data.fire_2022_mal_train, '\t')
mal_val = pd.read_csv(data.fire_2022_mal_val, '\t')

In [8]:
lengths_tamil_train = []
lengths_tamil_val = []
for i in tamil_train.text:
    lengths_tamil_train.append(len(i.split(' ')))

for i in tamil_val.text:
    lengths_tamil_val.append(len(i.split(' ')))

print(f"Average word count tamil_train: {sum(lengths_tamil_train) / (len(lengths_tamil_train))}")
print(f"Average word count tamil_val: {sum(lengths_tamil_val) / (len(lengths_tamil_val))}")

Average word count tamil_train: 10.864342607134844
Average word count tamil_val: 10.847299343765775


In [9]:
lengths_kan_train = []
lengths_kan_val = []
for i in kan_train.text:
    lengths_kan_train.append(len(i.split(' ')))

for i in kan_val.text:
    lengths_kan_val.append(len(i.split(' ')))

print(f"Average word count kan_train: {sum(lengths_kan_train) / (len(lengths_kan_train))}")
print(f"Average word count kan_val: {sum(lengths_kan_val) / (len(lengths_kan_val))}")

Average word count kan_train: 8.031712813908564
Average word count kan_val: 7.992764109985528


In [10]:
lengths_mal_train = []
lengths_mal_val = []
for i in mal_train.text:
    lengths_mal_train.append(len(i.split(' ')))

for i in mal_val.text:
    lengths_mal_val.append(len(i.split(' ')))

print(f"Average word count kan_train: {sum(lengths_mal_train) / (len(lengths_mal_train))}")
print(f"Average word count kan_val: {sum(lengths_mal_val) / (len(lengths_mal_val))}")

Average word count kan_train: 10.878839375629406
Average word count kan_val: 11.016987542468856


In [1]:
import torch
from tqdm import tqdm
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import numpy as np
from sklearn.metrics import classification_report
from transformers import TOKENIZER_MAPPING, AutoModelForSequenceClassification, AutoTokenizer, AdamW, get_linear_schedule_with_warmup, XLMRobertaTokenizer, XLMRobertaForSequenceClassification
import os
from util import create_output

TOKENIZER_NAME = "sentence-transformers/paraphrase-xlm-r-multilingual-v1"
MODEL_NAME = "sentence-transformers/paraphrase-xlm-r-multilingual-v1"
LEARNING_RATE = 3e-5

OUTPUT_FILE = "paraphrase-roberta-tamil.md"

EPOCHS = 4
BATCH_SIZE = 24
os.environ["CUDA_VISIBLE_DEVICES"]="5"

if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())    
    print(f'We will use the GPU:{torch.cuda.get_device_name()} ({device})')

else:
    print('NO GPU AVAILABLE ERROR')
    device = torch.device("cpu")
   

tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=5, output_attentions=True)
model.to(device)
optimizer = AdamW(model.parameters(), lr = LEARNING_RATE)

data = Dataset()
tam_train_2022, tam_val_2022, kannada_train_2022, kannada_val_2022, malayalam_train_2022, malayalam_val_2022 = data.get_fire_2022_dataset(tokenizer)

There are 1 GPU(s) available.
We will use the GPU:Tesla V100-SXM2-32GB (cuda)


Some weights of the model checkpoint at sentence-transformers/paraphrase-xlm-r-multilingual-v1 were not used when initializing XLMRobertaForSequenceClassification: ['pooler.dense.bias', 'pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at sentence-transformers/paraphrase-xlm-r-multilingual-v1 and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_pro

In [11]:
tam_train_lengths = []
tam_val_lengths = []

for i in tam_train_2022.tensors[1]:
    tam_train_lengths.append(int(i.count_nonzero()))

for i in tam_val_2022.tensors[1]:
    tam_val_lengths.append(int(i.count_nonzero()))

print(f"Tam train avg tokens: {sum(tam_train_lengths) / len(tam_train_lengths)}")
print(f"Tam val avg tokens: {sum(tam_val_lengths) / len(tam_val_lengths)}")


Tam train avg tokens: 23.188607807942564
Tam val avg tokens: 23.042402826855124


In [12]:
kan_train_lengths = []
kan_val_lengths = []

for i in kannada_train_2022.tensors[1]:
    kan_train_lengths.append(int(i.count_nonzero()))

for i in kannada_val_2022.tensors[1]:
    kan_val_lengths.append(int(i.count_nonzero()))

print(f"kan train avg tokens: {sum(kan_train_lengths) / len(kan_train_lengths)}")
print(f"kan val avg tokens: {sum(kan_val_lengths) / len(kan_val_lengths)}")

kan train avg tokens: 17.666934964584676
kan val avg tokens: 17.795947901591894


In [13]:
mal_train_lengths = []
mal_val_lengths = []

for i in malayalam_train_2022.tensors[1]:
    mal_train_lengths.append(int(i.count_nonzero()))

for i in malayalam_val_2022.tensors[1]:
    mal_val_lengths.append(int(i.count_nonzero()))

print(f"mal train avg tokens: {sum(mal_train_lengths) / len(mal_train_lengths)}")
print(f"mal val avg tokens: {sum(mal_val_lengths) / len(mal_val_lengths)}")


mal train avg tokens: 25.728222557905337
mal val avg tokens: 25.405436013590034
